In [1]:
import requests, zipfile, io, re, pandas

api = 'https://api.earthref.org/v1/MagIC/{}'

### Given a MagIC contribution ID, download the MagIC contribution text file if it is public

In [2]:
# Try to download the contribution text for an invalid ID
response = requests.get(api.format('download'), params={'id': 'a'})
print('Error:', response.json()['err'][0]['message'], '\n')

# Try to download the contribution text for a could valid, but nonexistant, IDs
response = requests.get(api.format('download'), params={'id': [1000, 1001]})
if (response.status_code == 204):
    print('Public contribution IDs 1000 and 1001 are not found in MagIC', '\n')
else:
    print('Error:', response.json()['err'][0]['message'], '\n')

# Download the contribution text for a valid ID and print the first 200 characters
contribution_id = 16761
response = requests.get(api.format('download'), params={'id': contribution_id})
if (response.status_code == 200):
    contribution_zip = zipfile.ZipFile(io.BytesIO(response.content))
    contribution_text = io.TextIOWrapper(contribution_zip.open('{}/magic_contribution_{}.txt'.format(contribution_id, contribution_id))).read()
    print(contribution_text[0:200], '\n')
elif (response.status_code == 204):
    print('Public contribution ID = "{}" not found in MagIC'.format(contribution_id), '\n')
else:
    print('Error:', response.json()['err'][0]['message'], '\n')

# Get the ID for the latest public MagIC contribution
latest_contribution_id_response = requests.get(api.format('search/contributions?n_max_rows=1'))
if (latest_contribution_id_response.status_code == 200):
    latest_contribution_id = latest_contribution_id_response.json()['results'][0]['id']
elif (latest_contribution_id_response.status_code == 204):
    print('Latest contribution ID not retrieved in MagIC', '\n')
else:
    print('Error:', latest_contribution_id_response.json()['err'][0]['message'], '\n')

# Get the contribution text for the ID and print the first 200 characters
latest_contribution_response = requests.get(api.format('download'), params={'id': latest_contribution_id})
if (latest_contribution_response.status_code == 200):
    latest_contribution_zip = zipfile.ZipFile(io.BytesIO(latest_contribution_response.content))
    latest_contribution_text = io.TextIOWrapper(latest_contribution_zip.open('{}/magic_contribution_{}.txt'.format(latest_contribution_id, latest_contribution_id))).read()
    print(latest_contribution_text[0:200], '\n')
elif (latest_contribution_response.status_code == 204):
    print('Public contribution ID = "{}" not found in MagIC'.format(latest_contribution_id), '\n')
else:
    print('Error:', latest_contribution_response.json()['err'][0]['message'], '\n')

Error: should be integer 

Public contribution IDs 1000 and 1001 are not found in MagIC 

tab delimited	contribution
id	version	timestamp	contributor	data_model_version	reference
16761	2	2020-02-23T23:03:45.034Z	@ltauxe	3.0	10.1016/J.EPSL.2014.12.034
>>>>>>>>>>
tab delimited	locations
loca 

tab delimited	contribution
id	version	timestamp	contributor	data_model_version	reference
16724	1	2020-02-27T15:55:11.037Z	@andrebie	3.0	10.1029/2018TC005285
>>>>>>>>>>
tab delimited	locations
location 



### Given a DOI, download the MagIC contribution text file for the latest 2 public versions

In [3]:
# Get the contribution text for each matching contribution and print the first 200 characters
reference_doi = '10.1016/J.PEPI.2017.05.009'
response = requests.get(api.format('download'), params={'doi': reference_doi, 'n_max_contributions': 2})
if (response.status_code == 200):
    contribution_zip = zipfile.ZipFile(io.BytesIO(response.content))
    for filename in contribution_zip.namelist():
        if (re.match(r'^\d+\/magic_contribution_\d+\.txt', filename)):
            contribution_text = io.TextIOWrapper(contribution_zip.open(filename)).read()
            print(contribution_text[0:200], '\n')
elif (response.status_code == 204):
    print('Public contribution with a reference DOI = "{}" not found in MagIC'.format(reference_doi), '\n')
else:
    print('Error:', response.json()['err'][0]['message'], '\n')

tab delimited	contribution
id	version	timestamp	contributor	data_model_version	reference	description
16595	9	2019-02-07T19:49:09.796Z	@najmagic	3.0	10.1016/J.PEPI.2017.05.009	Put 0.0 in the dir_dec of 

tab delimited	contribution
id	version	timestamp	contributor	data_model_version	reference	description
16583	8	2019-01-29T02:37:50.493Z	@najmagic	3.0	10.1016/J.PEPI.2017.05.009	Fixed problems caught wit 



### Get the 50 latest public sites in MagIC that mention basalt or sandstone

In [4]:
# Get the matching sites rows into a Pandas dataframe
response = requests.get(api.format('search/sites'), params={'query': 'basalt OR sandstone', 'n_max_rows': 50})
if (response.status_code == 200):
    sites = response.json()['results']
elif (response.status_code == 204):
    print('No sites in MagIC match the search parameters.', '\n')
else:
    print('Error:', response.json()['err'][0]['message'], '\n')

sites_df = pandas.DataFrame(sites)
sites_df.head(50)

,dir_n_samples,vgp_lat,lon,geologic_types,formation,dir_k,method_codes,geologic_classes,vgp_lon,citations,...,dir_inc,bed_dip,dir_alpha95,age_low,site,lithologies,dir_r,location,age_unit,result_quality
0,3,-70.4,255.35678,Sediment Layer,D1 Synorogenic Sediments,140.86,DE-FM:DE-RAN:FS-H:FS-LOC-GPS:GM-FOSSIL:LP-DC4:...,Sedimentary,294.5,10.24872/RMGJOURNAL.54.2.133,...,-40.1,0,58.2,60,DB0913,Sandstone:Siltstone:Mudstone:Muddy Sandstone,3,Leaf 6,Ma,g
1,3,-44.6,255.35681,Sediment Layer,D1 Synorogenic Sediments,39.39,DE-FM:DE-RAN:FS-H:FS-LOC-GPS:GM-FOSSIL:LP-DC4:...,Sedimentary,347.3,10.24872/RMGJOURNAL.54.2.133,...,-46,0,19.9,60,CB1739,Sandstone:Siltstone:Mudstone:Muddy Sandstone,2.95,Leaf 6,Ma,g
2,3,-79.3,255.35747,Sediment Layer,D1 Synorogenic Sediments,26.15,DE-FM:DE-RAN:FS-H:FS-LOC-GPS:GM-FOSSIL:LP-DC4:...,Sedimentary,160.2,10.24872/RMGJOURNAL.54.2.133,...,-58.3,0,24.6,60,CB1833,Sandstone:Siltstone:Mudstone,2.92,9776,Ma,g
3,3,-83.5,255.35718,Sediment Layer,D1 Synorogenic Sediments,28.48,DE-FM:DE-RAN:FS-H:FS-LOC-GPS:GM-FOSSIL:LP-DC4:...,Sedimentary,284.3,10.24872/RMGJOURNAL.54.2.133,...,-52.5,0,23.5,60,CB1830,Sandstone:Siltstone:Mudstone,2.93,9776,Ma,g
4,3,-63,255.35717,Sediment Layer,D1 Synorogenic Sediments,13.45,DE-FM:DE-RAN:FS-H:FS-LOC-GPS:GM-FOSSIL:LP-DC4:...,Sedimentary,319.8,10.24872/RMGJOURNAL.54.2.133,...,-41.7,0,35,60,CB1803,Sandstone:Siltstone:Mudstone,2.85,9776,Ma,g
5,3,31.9,255.3572,Sediment Layer,D1 Synorogenic Sediments,5.8,DE-FM:DE-RAN:FS-H:FS-LOC-GPS:GM-FOSSIL:LP-DC4:...,Sedimentary,163.9,10.24872/RMGJOURNAL.54.2.133,...,33.5,0,56.7,60,CB1831,Sandstone:Siltstone:Mudstone,2.66,9776,Ma,g
6,3,-79.8,255.35721,Sediment Layer,D1 Synorogenic Sediments,11.3,DE-FM:DE-RAN:FS-H:FS-LOC-GPS:GM-FOSSIL:LP-DC4:...,Sedimentary,358,10.24872/RMGJOURNAL.54.2.133,...,-59.5,0,38.5,60,CB1832,Sandstone:Siltstone:Mudstone,2.82,9776,Ma,g
7,3,87.1,255.35742,Sediment Layer,D1 Synorogenic Sediments,538.42,DE-FM:DE-RAN:FS-H:FS-LOC-GPS:GM-FOSSIL:LP-DC4:...,Sedimentary,145.3,10.24872/RMGJOURNAL.54.2.133,...,57.2,0,5.3,60,CB1802,Sandstone:Siltstone:Mudstone,3,9776,Ma,g
8,3,83.2,255.35732,Sediment Layer,D1 Synorogenic Sediments,330.92,DE-FM:DE-RAN:FS-H:FS-LOC-GPS:GM-FOSSIL:LP-DC4:...,Sedimentary,81,10.24872/RMGJOURNAL.54.2.133,...,51.4,0,6.8,60,CB1801,Sandstone:Siltstone:Mudstone,2.99,9776,Ma,g
9,3,82.2,255.35673,Sediment Layer,D1 Synorogenic Sediments,164.7,DE-FM:DE-RAN:FS-H:FS-LOC-GPS:GM-FOSSIL:LP-DC4:...,Sedimentary,121.4,10.24872/RMGJOURNAL.54.2.133,...,52.6,0,9.6,60,CB1745,Sandstone:Siltstone:Mudstone,2.99,9776,Ma,g
